In [23]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [24]:
import json
import numpy
import re
import pandas as pd
from collections import Counter
import requests
from bs4 import BeautifulSoup
from nltk import pos_tag
from nltk import RegexpParser
from fractions import Fraction
from transformers import pipeline
import tkinter as tk
from functools import partial 
# import pyttsx3
import tkinter as tk
# import customtkinter
# import speech_recognition as sr
global_dish = 'drumsticks'

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def get_recipe_details_selenium(dish):
    ingredient_list = []
    step_list = []
    level = '-'
    total_time = '-'
    prep_time = '-'
    cook_time = '-'
    servings = '-'

    dish_to_search = '-'.join(dish.split(' ')) + '-'
    URL = f"https://www.foodnetwork.com/search/{dish_to_search}"

    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(URL)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    recipe_result_list = soup.find_all('div',{'class':'m-MediaBlock__m-TextWrap'})
    recipe_links = []
    for i in recipe_result_list:
        if i.find('span',{'class':'m-Info__a-SubHeadline'}) is not None:
            is_result_recipe = i.find('span',{'class':'m-Info__a-SubHeadline'}).text.strip() == 'Recipe'
            if is_result_recipe:
                recipe_link_tag = i.find('h3',{'class':'m-MediaBlock__a-Headline'})
                recipe_links.append('https:' + recipe_link_tag.find('a')['href'])            

    recipe_link_url = recipe_links[0] if recipe_links else None
    print("菜谱链接:", recipe_link_url)

    if recipe_link_url is None:
        driver.quit()
        print("没有找到菜谱链接")
        return {}

    driver.get(recipe_link_url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    time_serving_details = soup.find_all('span',{'class':'o-RecipeInfo__a-Description'})
    if time_serving_details:
        text_list = [x.text.strip() for x in time_serving_details]
        if len(text_list) >= 6:  # inactive time
            level = text_list[0]
            total_time = text_list[1]
            prep_time = text_list[2] + '+' + text_list[3]
            cook_time = text_list[4]
            servings = text_list[5]
        elif len(text_list) >= 5:
            level = text_list[0]
            total_time = text_list[1]
            prep_time = text_list[2]
            cook_time = text_list[3]
            servings = text_list[4]
        else:
            level = text_list[0]
            total_time = text_list[1]
            cook_time = text_list[2]
            servings = text_list[3]

    ingredients = soup.find_all('span', {'class': 'o-Ingredients__a-Ingredient--CheckboxLabel'})
    ingredient_list = [ingredient.text.strip() for ingredient in ingredients]
    if ingredient_list:
        if ingredient_list[0]=='':
            ingredient_list = ingredient_list[1:]
    steps = soup.find_all('li',{'class':'o-Method__m-Step'})
    step_list = [step.text.strip() for step in steps]

    recipe = {
        'ingredient': ingredient_list,
        'step': step_list,
        'level': level,
        'total_time': total_time,
        'prep_time': prep_time,
        'cook_time': cook_time,
        'servings': servings
    }
    return recipe

result = get_recipe_details_selenium("drumsticks")
print(result)


菜谱链接: https://www.foodnetwork.com/recipes/melissa-darabian/buffalo-drumsticks-recipe-2041689
{'ingredient': ['Deselect All', '8 drumsticks', '2 tablespoons vegetable oil', '1 teaspoon chili powder', '1 teaspoon garlic powder', 'Salt and freshly ground black pepper', '1/4 cup (1/2 stick) unsalted butter, melted', '1/4 cup hot sauce', '1 tablespoon BBQ sauce'], 'step': ['For the chicken: Preheat the oven to 375 degrees F. Rinse and pat dry the drumsticks. Mix the oil, chili powder, garlic powder, and some salt and pepper in a medium bowl or resealable plastic bag. Toss the drumsticks in the seasoned oil to coat well. Bake the drumsticks on a wire rack placed on a baking sheet (so air circulates around chicken) until very tender and the skin is crisp, 50 minutes.', 'For the spicy sauce: Mix together the butter, hot sauce and BBQ sauce in a large bowl.', 'Toss the drumsticks in the spicy sauce until well coated. Place the drumsticks back on the rack and continue baking an additional 10 to 

In [26]:
def pos_tagging_ingredient(recipe,global_dish):
    pos_ingredient_dict = {}
    for ingredient in result['ingredient']:
        split_ingredient = ingredient.split()
        if global_dish not in pos_ingredient_dict:
            pos_ingredient_dict[global_dish] = []
        pos_ingredient_dict[global_dish].append(pos_tag(split_ingredient))
        
    return pos_ingredient_dict

In [27]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
s = 'how much olive oil'
tag=pos_tag(s.split(' '))
tag

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


[('how', 'WRB'), ('much', 'JJ'), ('olive', 'JJ'), ('oil', 'NN')]

In [28]:
#regex for ingredientQueries: how much, amount, quantity, how many, count of 
ingredientQueries=['much','amount','quantity','many','count']
search_nouns=[]
for ta in tag:
    if ta[1]=='NN':
        search_nouns.append(ta[0])
print(search_nouns)  

['oil']


In [29]:
#with POS-Searching the noun from the query in ingredient
se=0
if search_nouns[se] in ingredientQueries:
    while se<len(search_nouns):
        for ing in ingredient:
            if search_nouns[se] in ing.split():
                print(ing)                
        se+=1

In [30]:
ingredient = result['ingredient']
print(ingredient)

['Deselect All', '8 drumsticks', '2 tablespoons vegetable oil', '1 teaspoon chili powder', '1 teaspoon garlic powder', 'Salt and freshly ground black pepper', '1/4 cup (1/2 stick) unsalted butter, melted', '1/4 cup hot sauce', '1 tablespoon BBQ sauce']


In [31]:
se=0
while se<len(s.split()):
    for ing in ingredient:
        if s.split()[se] in ing.split():
            print(ing)                
    se+=1

2 tablespoons vegetable oil


In [32]:
def IngredientQuery(s):
    dish = global_dish
    pos_ingredient_dict = {}
    for ingredient in result['ingredient']:
        split_ingredient = ingredient.split()
        if dish not in pos_ingredient_dict:
            pos_ingredient_dict[dish] = []
        pos_ingredient_dict[dish].append(pos_tag(split_ingredient)) 
        
    ingredientQueries=['much','amount','quantity','many','count','step'] #Generic query words related to ingredient questions
    
    #let us extract all the nouns from the ingredient list and add them to the ingredientQueries list
    ingredient=recipe[dish]['ingredient'] 
    for ingr in ingredient:
        wordIng=ingr.split()
        tagIng=pos_tag(wordIng)
        for tagIn in tagIng:
            if tagIn[1]=='NN' or tagIn[1]=='NNS' or tagIn[1]=='NNP': #filter on the basis of Noun 
                ingredientQueries.append(tagIn[0])
                
    #Now let us extract all the nouns from the asked question
    search_nouns=[]
    tag=pos_tag(s.split(' '))
    for ta in tag:
        if ta[1]=='NN':
            search_nouns.append(ta[0])
    
    #IF the nouns from question matches the nouns from ingredientQueries list, return the ingredient with description
    se=0
    while se<len(search_nouns):
        if search_nouns[se] in ingredientQueries:
            for ing in ingredient:
                if search_nouns[se] in ing.split():
                    return(ing)                
        se+=1

In [33]:
qa_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [34]:
def scale_ingredients(global_dish,scale_ingredient_by,pos_ingredient_dict):
    pos_ingredient_dict_scaled = pos_ingredient_dict
    counter1 = 0
    for element in pos_ingredient_dict[global_dish]:
        counter2 = 0
        for tags in element:
            if tags[1] == 'CD':
                pos_ingredient_dict_scaled[global_dish][counter1][counter2] = [float(Fraction(tags[0]))*scale_ingredient_by,'CD']
            counter2+=1
        counter1+=1
    return pos_ingredient_dict_scaled

In [35]:
def print_ingredients(global_dish,ingredients):
    for element in ingredients[global_dish]:
        ingredient = ''
        for tags in element:
            ingredient += str(tags[0]) + ' '
        print(ingredient)

In [36]:
pos_ingredient_dict = {}
pos_ingredient_dict_scaled = {}
scale_ingredient_by = 12

import re
serving_text = result['servings']
serving_match = re.search(r'\d+', serving_text)
if serving_match:
    servings = int(serving_match.group())
else:
    servings = 1

if servings != 0:
    pos_ingredient_dict = pos_tagging_ingredient(result, global_dish)
    print_ingredients(global_dish, pos_ingredient_dict)
    print()
    pos_ingredient_dict_scaled = scale_ingredients(global_dish, (scale_ingredient_by/servings), pos_ingredient_dict)
    print_ingredients(global_dish, pos_ingredient_dict_scaled)


Deselect All 
8 drumsticks 
2 tablespoons vegetable oil 
1 teaspoon chili powder 
1 teaspoon garlic powder 
Salt and freshly ground black pepper 
1/4 cup (1/2 stick) unsalted butter, melted 
1/4 cup hot sauce 
1 tablespoon BBQ sauce 

Deselect All 
96.0 drumsticks 
24.0 tablespoons vegetable oil 
12.0 teaspoon chili powder 
12.0 teaspoon garlic powder 
Salt and freshly ground black pepper 
3.0 cup (1/2 stick) unsalted butter, melted 
3.0 cup hot sauce 
12.0 tablespoon BBQ sauce 


In [38]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
import nest_asyncio

# 初始化设置
nest_asyncio.apply()
Settings.llm = Ollama(model="qwen:7b", request_timeout=600.0)
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")
Settings.chunk_size = 1024  # 适合菜谱文本的长度

# 示例菜谱数据
recipe_data = {
    "Pasta": {
        "ingredient": ["200g pasta", "2 cloves garlic", "100ml olive oil"],
        "step": ["Boil pasta 8 mins", "Chop garlic", "Fry garlic", "Mix all"],
        "metadata": {
            "cuisine": "Italian",
            "cook_time": "20 mins",
            "difficulty": "Easy"
        }
    },
    "Kung Pao Chicken": {
        "ingredient": ["500g chicken", "50g peanuts", "3 dried chilies"],
        "step": ["Marinate chicken", "Stir-fry chilies", "Add peanuts"],
        "metadata": {
            "cuisine": "Chinese",
            "cook_time": "30 mins",
            "difficulty": "Medium"
        }
    }
}

# 文档预处理
def prepare_recipe_documents(data):
    documents = []
    for name, details in data.items():
        text = f"""
        Recipe: {name}
        Cuisine: {details['metadata']['cuisine']}
        Cook Time: {details['metadata']['cook_time']}
        Difficulty: {details['metadata']['difficulty']}
        
        Ingredients:
        {', '.join(details['ingredient'])}
        
        Steps:
        {'; '.join(details['step'])}
        """
        documents.append(Document(
            text=text,
            metadata={"recipe_name": name, **details["metadata"]}
        ))
    return documents

# 构建索引
documents = prepare_recipe_documents(recipe_data)
index = VectorStoreIndex.from_documents(documents)

In [40]:
# 简单查询引擎
basic_engine = index.as_query_engine()

def query_recipe(question):
    response = basic_engine.query(question)
    print(f"Q: {question}\nA: {response}\n")

# 示例查询
query_recipe("如何制作意大利面？")
query_recipe("宫保鸡丁需要哪些原料？")
query_recipe("Pizza需要哪些原料?")
query_recipe("请比较Pasta和宫保鸡丁的制作难度")
query_recipe("推荐一个制作时间短的菜谱")

Q: 如何制作意大利面？
A: 要制作意大利面，按照下面的步骤操作即可：

1. **准备材料**：购买200克意大利面条（Pasta），2瓣大蒜（Chopped Garlic），100毫升橄榄油（Olive Oil）。

2. **煮面**：将大锅加水，水量要比面条多一些，然后将火调至中等，等待水沸腾后（约8分钟）放入意大利面条。

3. **炒蒜**：当面条开始变软时，捞出面条备用。这时将切好的大蒜放入热油中（不要让油溅起），用中小火慢慢炒至金黄色。

4. **混合**：最后，将炒好的蒜末和煮好的意大利面条一起倒入锅中，用剩下的橄榄油轻轻拌匀，让面条充分吸收蒜香味即可上桌享用了。

Q: 宫保鸡丁需要哪些原料？
A: 宫保鸡丁需要以下原料：

- 500克鸡肉（可以选择去骨鸡胸肉或鸡腿肉）
- 50克炒熟的花生米（也称为“花生酱”）
- 3个干辣椒剪成小段

除此之外，制作宫保鸡丁还需要一些调料和蔬菜：

- 3瓣大蒜切片
- 2个大洋葱切成小块
- 100毫升生抽酱油
- 50毫升老抽酱油（用于调色）
- 50克白糖（用来中和辣味并增加甜度）
- 2茶匙淀粉水（用于勾芡）

按照上述原料和步骤，您可以成功制作宫保鸡丁了。

Q: Pizza需要哪些原料?
A: 根据提供的烹饪背景信息，制作 pizza 需要以下原料：

1. 面团：通常用高筋面粉，酵母发酵，揉成有弹性的面团。

2. 番茄酱：基础是番茄和调料，用于涂抹在面团上作为底料。

3. 奶酪（如马苏里拉奶酪）：切成薄片或刨丝后放在披萨上，烤制时融化并增添风味。

4. 其他配料：可以根据个人喜好添加各种食材，如蔬菜（如菠菜、洋葱）、香肠、火腿等。

Q: 请比较Pasta和宫保鸡丁的制作难度
A: Pasta and宫保鸡丁的制作难度可以这样比较：

Pasta：
- Difficulty: Easy
According to the provided context, Pasta is described as an easy recipe to make.

宫保鸡丁：
- Difficulty: Medium
The context mentions that宫保鸡丁 has a difficulty level of Medium. This indicates that the recip

In [41]:
from llama_index.core.memory import ChatMemoryBuffer

# 带记忆的聊天引擎
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
chat_engine = index.as_chat_engine(
    chat_mode="best",
    memory=memory,
    verbose=True
)

def recipe_chat():
    print("菜谱助手已就绪（输入'退出'结束）")
    while True:
        query = input("\n您的问题：")
        if query.lower() in ['退出', 'exit']:
            break
        response = chat_engine.chat(query)
        print(f"助手：{response}")

# 启动对话
recipe_chat()

菜谱助手已就绪（输入'退出'结束）

您的问题：How Do you make spaghetti?
> Running step 2fdaf3ea-942f-47a7-80bc-503b1dac96cf. Step input: How Do you make spaghetti?
Thought: I can answer without using any more tools. I'll use English to provide a recipe for making spaghetti.
Answer: To make spaghetti, follow these steps:

1. **Ingredients:**
   - 2 cups all-purpose flour
   - 3 large eggs
   - 1 teaspoon salt
   - 4-5 quarts water

2. **Preparing the Pasta Dough:**
   - In a large mixing bowl, combine the flour and salt.
   - Make a well in the center of the flour mixture.
   - Crack the eggs into the well.
   - Using a fork or your hands, gradually mix the flour and eggs together until a shaggy dough forms. (Don't overwork the dough.)

3. **Kneading the Pasta Dough:**
   - On a floured surface, gently roll out the dough to a thickness of about 1/8 inch.
   - Use a pasta cutter or a sharp knife to cut the dough into thin, spaghetti-like strips.

4. **Cooking the Spaghetti:**
   - Bring the water to a rollin

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

def rerank_recipes(query, candidates, embed_model):
    query_embed = embed_model._get_text_embedding(query)
    doc_embeddings = [embed_model._get_text_embedding(node.text) for node in candidates]
    scores = cosine_similarity([query_embed], doc_embeddings)[0]
    return sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)

# 使用示例
query = "chicken"
nodes = basic_engine.retrieve(query)
ranked = rerank_recipes(query, nodes, Settings.embed_model)
for node, score in ranked[:3]:
    print(f"{node.metadata['recipe_name']} (匹配度: {score:.2f})")

Kung Pao Chicken (匹配度: 0.65)
Pasta (匹配度: 0.45)


In [43]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor, KeywordExtractor

# 1. 创建文本分割器
text_splitter = TokenTextSplitter(
    separator=" ",
    chunk_size=512,
    chunk_overlap=20
)

# 2. 创建内容提取器
title_extractor = TitleExtractor(nodes=5)
keyword_extractor = KeywordExtractor(keywords=5)

# 3. 正确初始化pipeline（注意不是直接调用）
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        title_extractor,
        keyword_extractor
    ]
)

# 4. 运行pipeline（这才是正确的调用方式）
nodes = pipeline.run(documents=documents)  # 注意使用run()方法

# 查看处理结果
print(f"生成 {len(nodes)} 个节点")
print("示例节点关键词:", nodes[0].metadata["excerpt_keywords"])

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.51s/it]

生成 2 个节点
示例节点关键词: 1. Italian Pasta Recipe
2. Easy Garlic Olive Oil Pasta
3. Classic Italian Cook Time 20 mins
4. Beginner-friendly Pasta Meal Idea
5. Healthy Italian Dinner Option


In [44]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor, KeywordExtractor

# 1. 创建文本分割器
text_splitter = TokenTextSplitter(
    separator=" ",
    chunk_size=512,
    chunk_overlap=20
)

# 2. 创建内容提取器
title_extractor = TitleExtractor(nodes=5)
keyword_extractor = KeywordExtractor(keywords=5)

# 3. 正确初始化pipeline（注意不是直接调用）
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        title_extractor,
        keyword_extractor
    ]
)

# 4. 运行pipeline（这才是正确的调用方式）
nodes = pipeline.run(documents=documents)  # 注意使用run()方法

# 查看处理结果
print(f"生成 {len(nodes)} 个节点")
print("示例节点关键词:", nodes[0].metadata["excerpt_keywords"])

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.92s/it]

生成 2 个节点
示例节点关键词: 1. Italian Pasta Recipe
2. Easy Garlic Olive Oil Pasta
3. Step-by-Step Italian Pasta Cooking
4. Gluten-Free Italian Pasta Option
5. Family-Friendly Italian Pasta Dinner


In [45]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor, KeywordExtractor
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core import get_response_synthesizer
# 1. 创建文本分割器
text_splitter = TokenTextSplitter(
    separator=" ",
    chunk_size=512,
    chunk_overlap=20
)

# 2. 创建内容提取器
title_extractor = TitleExtractor(nodes=5)
keyword_extractor = KeywordExtractor(keywords=5)

# 3. 正确初始化pipeline（注意不是直接调用）
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        title_extractor,
        keyword_extractor
    ]
)

# 4. 运行pipeline（这才是正确的调用方式）
nodes = pipeline.run(documents=documents)  # 注意使用run()方法

# 查看处理结果
print(f"生成 {len(nodes)} 个节点")
print("示例节点关键词:", nodes[0].metadata["excerpt_keywords"])

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.88s/it]

生成 2 个节点
示例节点关键词: 1. Italian Pasta Recipe
2. Easy Garlic Olive Oil Pasta
3. Simple Italian Dinner Option
4. 20-Minute Pasta Cook Time
5. Family-Friendly Italian Pasta Dish


In [57]:
# # Define transformations
# text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)
# title_extractor = TitleExtractor(nodes=5)
# kw_extractor = KeywordExtractor(keywords=10)

from llama_index.core.vector_stores import MetadataFilters

kw_extractor=KeywordExtractor(keywords=10)
# 将pipeline运用到document中，并向量化index
index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter, title_extractor, kw_extractor]
)

# 然后实现query中的关键词提炼，并匹配文档中的关键词

# 定义具有记忆的查询函数
memory_buffer = ChatMemoryBuffer.from_defaults(token_limit=3900)
class RAGQueryEngine(CustomQueryEngine):
    """RAG Query Engine"""
    
    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    memory: memory_buffer

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        response_obj = self.response_synthesizer.synthesize(query_str, nodes)
        return response_obj

# 定义使用的模型
mymodel = "qwen:7b"
llm = Ollama(model=mymodel, request_timeout=600.0)
def get_keywords_from_llama(query):
    #设置系统prompt限制LLM按照要求输出
    system_prompt = (
        f"你是一个专业的关键词提取助手。\n"
        f"请根据用户的以下问题或文本，提取出最多3个关键词!!!一定注意回答的每个关键词都不要超过3个中文字!!\n"
        f"如果你输出的答案中每个关键词超过3个字，请检查，然后再输出一个满足要求的结果给我\n"
        f"问题或文本: {query}\n"
        f"你的答案：关键词:"
    )
    formatted_prompt = system_prompt.format(query=query)
    response = ""
    while not response.startswith("关键词: "):
        response = llm.complete(formatted_prompt).text
        if not response.startswith("关键词: "):
            continue
    #处理LLM输出关键词文本的结构，变成可处理的统一形式
    keywords_part = response[len("关键词: "):].strip().rstrip('。')
    separators = ['、', '，', ',']
    chosen_separator = None
    for sep in separators:
        if sep in keywords_part:
            chosen_separator = sep
            break
    if chosen_separator is None:
        keywords = [keywords_part] if keywords_part.strip() else []
    else:
        keywords = [keyword.strip() for keyword in keywords_part.split(chosen_separator) if keyword.strip()]
    return keywords

# 将输入的字符串按每两个中文字符分割成列表中的元素
def split_into_pairs(text):
    """
    将输入的字符串按每两个中文字符分割成列表中的元素。
    如果最后一个分段不足两个字符，则保持原样。
    """
    pairs = []
    length = len(text)
    for i in range(0, length, 2):
        pairs.append(text[i:min(i+2, length)])
    return pairs

# 定义关键词提取函数
def qs(query):
    keywords = get_keywords_from_llama(query)
    
    # 将每个关键词进一步分割为每两个字符的片段
    split_keywords = [split_into_pairs(keyword) for keyword in keywords]
    
    flattened_keywords = [item for sublist in split_keywords for item in sublist]
    
    return flattened_keywords

# 关键词提取示例
qs("我想吃西餐，可以吃宫保鸡丁吗？")

# 实现关键词匹配的查询
def query_with_keywords_matched(query):
    keywords = get_keywords_from_llama(query)
    filters_list = [
        MetadataFilter(key="excerpt_keywords", value=keyword, operator="contains")
        for keyword in keywords
    ]
    filters = MetadataFilters(filters=filters_list)
    retriever = index.as_retriever(
        filters=filters,
    )
    synthesizer = get_response_synthesizer(response_mode="compact")

    query_engine = RAGQueryEngine(
        retriever=retriever, response_synthesizer=synthesizer, memory=memory_buffer
    )
    response = query_engine.query(query)
    print(response)

# 关键词匹配查询示例
query_with_keywords_matched("我想吃西餐，可以吃宫保鸡丁吗？")

# 对比不使用关键词匹配的查询（设置top_k）
retriever = index.as_retriever(
    top_k = 8,
)

synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RAGQueryEngine(
    retriever=retriever, response_synthesizer=synthesizer, memory=memory_buffer
)

response1 = query_engine.query(f"我想吃西餐，可以吃宫保鸡丁吗？")
print(str(response1))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.82s/it]


Empty Response
宫保鸡丁虽然是川菜，而不是传统的西餐，但它属于中式菜肴，可以在某些西餐厅菜单上看到。所以如果你想在西餐厅尝试，可以选择一些改良版的中式菜品，但不一定会有正宗的宫保鸡丁供应。


In [58]:
def multi_turn_recipe_chat():
    memory = ChatMemoryBuffer.from_defaults()
    chat_engine = index.as_chat_engine(
        chat_mode="condense_plus_context",
        memory=memory,
        system_prompt=(
            "您是一位专业厨师助手，请根据菜谱数据库回答用户问题。\n"
            "回答时请：\n"
            "1. 明确说明菜谱名称\n"
            "2. 分步骤说明制作方法\n"
            "3. 给出烹饪小贴士\n"
            "4. 主动询问是否需要补充信息"
        )
    )
    
    while True:
        query = input("\n您的烹饪问题：")
        if query.lower() == '退出':
            break
            
        response = chat_engine.chat(query)
        print(f"\n厨师助手：{response}")
        
        # 自动生成后续问题
        follow_up = chat_engine.chat(
            "基于刚才的回答，生成一个能帮助用户更好理解烹饪过程的专业问题"
        )
        print(f"\n[建议提问]: {follow_up}")

# 启动对话
multi_turn_recipe_chat()


您的烹饪问题：During the preparation of a tender Kung Pao chicken, how do you handle the chicken properly to ensure the best taste?

厨师助手：To prepare tender Kung Pao chicken, handling it properly is crucial for flavor and texture. Here are some steps to follow:

1. **选购新鲜鸡肉**: Choose high-quality, boneless chicken breasts or thighs. Fresh meat will cook evenly and retain more moisture.

2. **Cleaning and trimming**: Thoroughly wash the chicken under cold running water to remove any surface bacteria. Trim off any fat or visible skin.

3. **Marinating (optional)**: If you prefer a more tender texture, marinate the chicken in a mixture of soy sauce, rice vinegar, sugar, garlic, and ginger for at least an hour, or even overnight in the refrigerator.

4. **Cooking**: After marinating, either stir-fry the chicken in a wok with diced bell peppers, dried chilies, peanuts, and a generous amount of oil until it's crispy and golden brown, or bake it in a preheated oven at 200°C (400°F) for about 25-30 m

In [59]:
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=64)  # 文本分割器
title_extractor = TitleExtractor()  # 标题提取器

optimized_index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter, title_extractor, kw_extractor]  # 三级处理流水线
)

from tenacity import retry, stop_after_attempt

@retry(stop=stop_after_attempt(3))  # 失败时自动重试最多3次
def robust_query(question):
    return optimized_index.as_query_engine().query(question)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.32s/it]


In [60]:
def select_recipe_by_criterion(query_result, criterion_key, criterion_value):
    for recipe in query_result.values():
        if recipe.get(criterion_key) == criterion_value:
            return recipe
    return None

In [61]:
query_result = robust_query("推荐一个菜谱")
query_result.metadata

{'97c7b000-33ea-4b82-a32e-43ec26b9e417': {'recipe_name': 'Kung Pao Chicken',
  'cuisine': 'Chinese',
  'cook_time': '30 mins',
  'difficulty': 'Medium',
  'document_title': '"Delicious and Nutritious Complete Kung Pao Chicken Recipe with a Unique Nutty Twist"',
  'excerpt_keywords': '1. Kung Pao Chicken recipe\n2. Chinese cuisine扭转会意的\n3. 30分钟简易家庭版\n4. 花椒与辣椒的独特融合\n5. 鸡肉、花生和辣椒的三重奏\n6. 简单几步教你做正宗\n7. 健康营养，美味又快捷的家庭菜'},
 'f82e7d4c-2e10-4141-8975-14a193dd2eec': {'recipe_name': 'Pasta',
  'cuisine': 'Italian',
  'cook_time': '20 mins',
  'difficulty': 'Easy',
  'document_title': '"Easy Italian Pasta Recipe: Flavorful Garlic and Olive Oil Preparation"',
  'excerpt_keywords': 'Keywords: Italian Pasta Recipe, Easy Cooking, Garlic Flavor, Olive Oil Preparation, Pasta Ingredients, Cooking Steps.'}}

In [63]:
query_result = robust_query("推荐一个菜谱")
actual_result = query_result.metadata
selected_recipe = select_recipe_by_criterion(actual_result, "cuisine", "Chinese")
if selected_recipe:
    keywords_lines = selected_recipe['excerpt_keywords'].split('\n')
    output = (
        f"推荐菜谱：{selected_recipe['document_title']}\n"
        f"难度：{selected_recipe.get('difficulty', '未知')}\n"
        f"关键步骤：\n" +
        '\n'.join(keywords_lines)  # 使用字符串连接而不是 f-string
    )
    print(output)
else:
    print("未找到符合条件的菜谱")

推荐菜谱："Delicious and Nutritious Complete Kung Pao Chicken Recipe with a Unique Nutty Twist"
难度：Medium
关键步骤：
1. Kung Pao Chicken recipe
2. Chinese cuisine扭转会意的
3. 30分钟简易家庭版
4. 花椒与辣椒的独特融合
5. 鸡肉、花生和辣椒的三重奏
6. 简单几步教你做正宗
7. 健康营养，美味又快捷的家庭菜


In [64]:
selected_recipe

{'recipe_name': 'Kung Pao Chicken',
 'cuisine': 'Chinese',
 'cook_time': '30 mins',
 'difficulty': 'Medium',
 'document_title': '"Delicious and Nutritious Complete Kung Pao Chicken Recipe with a Unique Nutty Twist"',
 'excerpt_keywords': '1. Kung Pao Chicken recipe\n2. Chinese cuisine扭转会意的\n3. 30分钟简易家庭版\n4. 花椒与辣椒的独特融合\n5. 鸡肉、花生和辣椒的三重奏\n6. 简单几步教你做正宗\n7. 健康营养，美味又快捷的家庭菜'}